# Case Study 1 : Collecting Data from Twitter

Due Date: 2/8/2021, **BEFORE the beginning of class at 11:00am**

## **NOTE: There are *always* last minute issues submitting the case studies.  DO NOT WAIT UNTIL THE LAST MINUTE!**

<img src="https://upload.wikimedia.org/wikipedia/en/thumb/9/9f/Twitter_bird_logo_2012.svg/220px-Twitter_bird_logo_2012.svg.png">

**TEAM Members:** Please EDIT this cell and add the names of all the team members in your team

    member 1
    
    member 2
    
    ...

**Suggested Readings:** 
* Chapter 1 and Chapter 9 of the book "Mining the Social Web" can help a lot if you get stuck. 
* In fact, it is intentional that many of these questions can be answered directly from there (except for question 4)!
* The idea is to ease you into the case studies :-)

**Don't forget!**
* You will need to install the python-twitter library to access the Twitter API
 * pip install twitter


** NOTE **
* **Please don't forget to save the notebook frequently when working in IPython Notebook, otherwise the changes you made can be lost**.

# Problem 1 (20 points): Sampling Twitter Data with either Search or Streaming API about a certain topic

* Select a topic that you are interested in, for example, "WPI" or "Lady Gaga"
* Use Twitter API to sample a collection of tweets about this topic. (It would be recommended that the number of tweets should be larger than 200, but smaller than 1 million.
* Store the tweets you downloaded into a local file (txt file or json file) 

In [4]:
import twitter
#---------------------------------------------
# Define a Function to Login Twitter API
def oauth_login():
    # Prof. Paffenroth has a developer account for the class.  He will provide the Twitter access tokens for
    # each team
    # See https://developer.twitter.com/docs/auth/oauth for more information 
    # on Twitter's OAuth implementation.
    
    CONSUMER_KEY = 'oVek8jmhI4sz8SbKnT1e6UsIh'
    CONSUMER_SECRET ='3eGB6ip82K3XowIDEmz9JXIbRc2uFSlHvjPJ8Lmkf3HVIdlmhM'
    OAUTH_TOKEN = '571213367-H0r2WfySE7L4vgUJHv4YLW2vZGLa7TEPP7iWsTNu'
    OAUTH_TOKEN_SECRET = 'uKcYixkDRaCUvaP2oqH9a9fIaIB9nf4NKz32iqo2t7B7K'

    
    auth = twitter.oauth.OAuth(OAUTH_TOKEN, OAUTH_TOKEN_SECRET,
                               CONSUMER_KEY, CONSUMER_SECRET)
    
    twitter_api = twitter.Twitter(auth=auth)
    return twitter_api

twitter_api = oauth_login()

#----------------------------------------------
# Your code starts here
#   Please add comments or text cells in between to explain the general idea of each block of the code.
#   Please feel free to add more cells below this cell if necessary


### Report some statistics about the tweets you collected 

* The topic of interest: < INSERT YOUR TOPIC HERE>


* The total number of tweets collected:  < INSERT THE NUMBER HERE>

*-----------------------

# Problem 2 (20 points): Analyzing Tweets and Tweet Entities with Frequency Analysis

**1. Word Count:** 
* Use the tweets you collected in Problem 1, and compute the frequencies of the words being used in these tweets. 
* Plot a table of the top 30 words with their counts

In [2]:
#----------------------------------------------
# Your code starts here
#   Please add comments or text cells in between to explain the general idea of each block of the code.
#   Please feel free to add more cells below this cell if necessary


**2. Find the most popular tweets in your collection of tweets**

Please plot a table of the top 10 tweets that are the most popular among your collection, i.e., the tweets with the largest number of retweet counts.


In [3]:
#----------------------------------------------
# Your code starts here
#   Please add comments or text cells in between to explain the general idea of each block of the code.
#   Please feel free to add more cells below this cell if necessary


**3. Find the most popular Tweet Entities in your collection of tweets**

Please plot a table of the top 10 hashtags, top 10 user mentions that are the most popular in your collection of tweets.

In [4]:
#----------------------------------------------
# Your code starts here
#   Please add comments or text cells in between to explain the general idea of each block of the code.
#   Please feel free to add more cells below this cell if necessary


# Problem 3 (20 points): Getting "All" friends and "All" followers of a popular user in twitter


* choose a popular twitter user who has many followers, such as "ladygaga".
* Get the list of all friends and all followers of the twitter user.
* Plot 20 out of the followers, plot their ID numbers and screen names in a table.
* Plot 20 out of the friends (if the user has more than 20 friends), plot their ID numbers and screen names in a table.

In [87]:
#----------------------------------------------
# Your code starts here
#   Please add comments or text cells in between to explain the general idea of each block of the code.
#   Please feel free to add more cells below this cell if necessary

import time
from prettytable import PrettyTable

# Get followers
friend_cursor = -1
follower_cursor = -1

user = "bryt9"

followers = []
friends = []
while friend_cursor != 0 and follower_cursor != 0:
    try:
        if follower_cursor != 0:
            res_follower = twitter_api.followers.list(screen_name=user, cursor=follower_cursor, count=200)
            followers += [{"screen_name": u["screen_name"], "id": u["id"]}for u in res_follower['users']]
            follower_cursor = res_follower["next_cursor"]
            
        if friend_cursor != 0:
            res_friends = twitter_api.friends.list(screen_name=user, cursor=friend_cursor, count=200)
            friends += [{"screen_name": u["screen_name"], "id": u["id"]}for u in res_friends['users']]
            friend_cursor = res_friends["next_cursor"]
    except twitter.api.TwitterHTTPError:
        print("Waiting...")
        time.sleep(900)
    
    


In [94]:
friends[0]

{'screen_name': 'tedlieu', 'id': 21059255}

In [115]:
def generate_user_table(data, title, length=20):
    table = PrettyTable(["Screen Name", "Id"])
    table.title = title
    
    if len(data) < length:
        length = len(data)
        
    for i in range(length):
        row = data[i]
        table.add_row([row["screen_name"], row["id"]])
    
    return table

In [118]:
from prettytable import PrettyTable

friends_table  = generate_user_table(friends, "Friends")
followers_table  = generate_user_table(followers, "Followers")

print(friends_table)
print(followers_table)

+---------------------------------------+
|                Friends                |
+-----------------+---------------------+
|   Screen Name   |          Id         |
+-----------------+---------------------+
|     tedlieu     |       21059255      |
|      wsbmod     | 1282418324228337665 |
|  OldMemeArchive | 1349856571988258826 |
|      POTUS      | 1349149096909668363 |
|    nookipedia   |      213478688      |
|     Pokemon     |       96879107      |
|   staceyabrams  |      216065430      |
|     jschlatt    |      4694011844     |
| ProgressBar202_ |      4685749051     |
|  LeftAccidental | 1303873858936737793 |
|   mcblockdaily  | 1324477660182007810 |
|   StrangestMp4  | 1194413925569159169 |
|  CallMeCarsonYT |      2349536690     |
|  MoistCr1TiKaL  |      559229566      |
|    jacksfilms   |       9989862       |
|    robber0540   |      3086280070     |
|   hasanthehun   |      326756275      |
|   KamalaHarris  |       30354991      |
|     JoeBiden    |        939091 

* Compute the mutual friends within the two groups, i.e., the users who are in both friend list and follower list, plot their ID numbers and screen names in a table

In [122]:
#----------------------------------------------
# Your code starts here
#   Please add comments or text cells in between to explain the general idea of each block of the code.
#   Please feel free to add more cells below this cell if necessary

mutual = []
for u in followers:
    for r in friends:
        if u['id'] == r["id"]:
            mutual.append(u)
            
mutual_table  = generate_user_table(mutual, "Mutual")

print(mutual_table)



+-----------------------------+
|            Mutual           |
+----------------+------------+
|  Screen Name   |     Id     |
+----------------+------------+
| ashleybaldw1n  | 3133724330 |
| srastevensb201 | 885519948  |
|    ralphpig    | 860254500  |
|    tang_meg    |  11726682  |
|    mmbtang     |  11699752  |
+----------------+------------+


*------------------------

# Problem 4 (20 points): Business question 

Run some additional experiments with your data to gain familiarity with the twitter data and twitter API.

* Come up with a business question that Twitter data could help answer.
* Decribe the business case.
* How could Twitter data help a company decide how to spend its resources.

In [9]:
#----------------------------------------------
# Your code starts here
#   Please add comments or text cells in between to explain the general idea of each block of the code.
#   Please feel free to add more cells below this cell if necessary





*-----------------
# Done

All set! 

** What do you need to submit?**

* **Notebook File**: Save this IPython notebook, and find the notebook file in your folder (for example, "filename.ipynb"). This is the file you need to submit. Please make sure all the plotted tables and figures are in the notebook. If you used "ipython notebook --pylab=inline" to open the notebook, all the figures and tables should have shown up in the notebook.


* **PPT Slides**: please prepare PPT slides (for 15 minutes' talk) to present about the case study . We will ask two teams which are randomly selected to present their case studies in class for this case study. 

Please compress all the files in a zipped file.


** How to submit: **

        Please submit through email to Prof. Paffenroth (rcpaffenroth@wpi.edu).

#### We auto-process the submissions so make sure your subject line is *exactly*:

### DS3010 Case Study 1 Team ??

#### where ?? is your team number.
        
** Note: Each team just needs to submits one submission **

# Grading Criteria:

**Totoal Points: 100**


---------------------------------------------------------------------------
**Notebook results:**
    Points: 80


    -----------------------------------
    Question 1:
    Points: 20
    -----------------------------------
    
    -----------------------------------
    Question 2:
    Points: 20
    -----------------------------------
        
    -----------------------------------
    Question 3:
    Points: 20
    -----------------------------------
  
    -----------------------------------
    Question 4:  Business question
    Points: 20
    -----------------------------------
        Novelty: 10
        Interestingness: 10
    -----------------------------------
    Come up with a business question and describe how Twitter data and a NoSQL database such as MongoDB call help you anser it.


---------------------------------------------------------------------------
**Slides (for 10 minutes of presentation): Story-telling**
    Points: 20


1. Motivation about the data collection, why the topic is interesting to you.
    Points: 5 

2. Communicating Results (figure/table)
    Points: 10 

3. Story telling (How all the parts (data, analysis, result) fit together as a story?)
    Points: 5 


